# Stage 13 Homework Starter — Productization

## Objective
Deploy your trained model as a **reusable, handoff-ready API or dashboard** and finalize your project for reproducibility and clarity.

## Steps
1. Create a mock, very basic analysis in a notebook.
2. Clean your notebook by removing exploratory cells and documenting your code.
3. Move reusable functions into `/src/`.
4. Load your trained model from Stage 12 or earlier stages.
5. Pickle/save the model and test reload.
6. Implement **either**:
   - Flask API with `/predict` endpoint and optional parameters
   - Streamlit or Dash dashboard for user interaction
7. Include:
   - Error handling for invalid inputs
   - `requirements.txt` for reproducibility
   - Documentation in `README.md`
8. Test your deployment locally and provide evidence.
9. Organize project folders and finalize notebooks for handoff.

## 1. Create mock, very basic analysis

In [5]:
# Stage 13: Productization - Complete Solution
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set up styling
sns.set(style='whitegrid', palette='husl')
plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10
np.random.seed(101)

print("=== STAGE 13: PRODUCTIZATION ===")
print("Creating deployable model and API...")

# Create project structure
project_dirs = ['model', 'src', 'deliverables', 'deliverables/images']
for dir_name in project_dirs:
    Path(dir_name).mkdir(exist_ok=True)

=== STAGE 13: PRODUCTIZATION ===
Creating deployable model and API...


In [2]:
# 1. Create mock, very basic analysis
print("\n1. Creating basic analysis...")

# Load data
data_path = Path('/Users/yuqingyan/Desktop/bootcamp_Yuqing_Yan/project/data/processed/aapl_2023_cleaned.csv')
df = pd.read_csv(data_path, index_col='date', parse_dates=['date'])

# Basic analysis
print(f"Dataset shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Columns: {list(df.columns)}")

# Calculate basic statistics
basic_stats = df.describe()
print("\nBasic statistics:")
print(basic_stats[['Open', 'High', 'Low', 'Close', 'Volume']])

print("Basic analysis complete.")


1. Creating basic analysis...
Dataset shape: (250, 5)
Date range: 2023-01-03 00:00:00 to 2023-12-29 00:00:00
Columns: ['Close', 'High', 'Low', 'Open', 'Volume']

Basic statistics:
             Open        High         Low       Close      Volume
count  250.000000  250.000000  250.000000  250.000000  250.000000
mean     0.639802    0.639005    0.640348    0.647775    0.269937
std      0.243458    0.240288    0.240809    0.237232    0.136470
min      0.000000    0.000000    0.000000    0.000000    0.000000
25%      0.489513    0.477833    0.497427    0.503253    0.182365
50%      0.683600    0.685394    0.681874    0.690101    0.238376
75%      0.838918    0.837336    0.832454    0.851838    0.319967
max      1.000000    1.000000    1.000000    1.000000    1.000000
Basic analysis complete.


## 2. Notebook Cleanup
Remove exploratory cells and document your code.

In [3]:
# 2. Notebook Cleanup
print("\n2. Cleaning notebook for handoff...")

# Remove exploratory cells and document code clearly
# This section contains only the essential, production-ready code

# Feature engineering (from previous stages)
df['price_range'] = df['High'] - df['Low']
df['close_ma_5_prev'] = df['Close'].shift(1).rolling(window=5, min_periods=5).mean()
df['ret'] = df['Close'].pct_change()
df['lag_1'] = df['ret'].shift(1)
df['roll_mean_5'] = df['ret'].shift(1).rolling(5, min_periods=5).mean()
df['roll_vol_20'] = df['ret'].shift(1).rolling(20, min_periods=20).std()

df_clean = df.dropna().copy()
print(f"Cleaned dataset shape: {df_clean.shape}")

print("Notebook cleaned and ready for handoff.")


2. Cleaning notebook for handoff...
Cleaned dataset shape: (226, 11)
Notebook cleaned and ready for handoff.


## 3. Move reusable functions to /src/
Create src/utils.py and store functions there.

In [4]:
# 3. Move reusable functions to /src/
print("\n3. Creating utility functions...")

utils_content = '''"""
Utility functions for AAPL stock analysis
"""
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler

def calculate_metrics(df):
    """Calculate basic descriptive statistics"""
    return df.describe()

def engineer_features(df):
    """Engineer features for stock analysis"""
    df_copy = df.copy()
    
    # Price-based features
    df_copy['price_range'] = df_copy['High'] - df_copy['Low']
    df_copy['close_ma_5_prev'] = df_copy['Close'].shift(1).rolling(window=5, min_periods=5).mean()
    
    # Return-based features
    df_copy['ret'] = df_copy['Close'].pct_change()
    df_copy['lag_1'] = df_copy['ret'].shift(1)
    df_copy['roll_mean_5'] = df_copy['ret'].shift(1).rolling(5, min_periods=5).mean()
    df_copy['roll_vol_20'] = df_copy['ret'].shift(1).rolling(20, min_periods=20).std()
    
    return df_copy.dropna()

def train_model(X, y):
    """Train a linear regression model"""
    model = LinearRegression()
    model.fit(X, y)
    return model

def evaluate_model(model, X, y):
    """Evaluate model performance"""
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    
    return {
        'mse': mse,
        'rmse': rmse,
        'r2': r2,
        'mae': mae,
        'predictions': y_pred
    }

def prepare_prediction_data(open_price, high_price, low_price, volume, close_ma_5):
    """Prepare data for prediction"""
    price_range = high_price - low_price
    return np.array([[open_price, high_price, low_price, volume, close_ma_5, price_range]])
'''

with open('src/utils.py', 'w') as f:
    f.write(utils_content)

print("✓ Created src/utils.py with utility functions")


3. Creating utility functions...
✓ Created src/utils.py with utility functions


## 4. Folder Structure Reminder

Ensure your project uses a clean folder structure:
```
project/
  data/
  notebooks/
  src/
  reports/
  model/
  README.md
```
For API/Dashboard: minimal example:
```
project/
    app.py
    model.pkl
    requirements.txt
    README.md
```

## Train and Save Final Model

In [6]:
# Train and Save Final Model
print("\n4. Training and saving final model...")

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Prepare features and target
features = ['Open', 'High', 'Low', 'Volume', 'close_ma_5_prev', 'price_range']
X = df_clean[features].values
y = df_clean['Close'].values

# Train-test split (time-aware)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
from sklearn.metrics import r2_score, mean_squared_error
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Model performance:")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.6f}")
print(f"Features used: {features}")

# Save model
with open('model/model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("✓ Model saved to model/model.pkl")


4. Training and saving final model...
Model performance:
R² Score: 0.9929
RMSE: 0.009755
Features used: ['Open', 'High', 'Low', 'Volume', 'close_ma_5_prev', 'price_range']
✓ Model saved to model/model.pkl


## 5. Pickle / Save Final Model

### TODO: Replace this with your trained model

In [7]:
# 5. Test Model Loading
print("\n5. Testing model loading...")

# Load model
with open('model/model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Test prediction
test_features = np.array([[0.5, 0.6, 0.4, 0.3, 0.55, 0.2]])
prediction = loaded_model.predict(test_features)
print(f"Test prediction: {prediction[0]:.6f}")

# Verify model is the same
print(f"Model coefficients: {loaded_model.coef_}")
print(f"Model intercept: {loaded_model.intercept_:.6f}")

print("✓ Model loading and prediction test successful")


5. Testing model loading...
Test prediction: 0.492512
Model coefficients: [-0.60234809  0.75898113  0.81505666 -0.01151354  0.01765203 -0.05607553]
Model intercept: 0.017235
✓ Model loading and prediction test successful


## 6. Flask API Starter

### TODO: Implement Flask endpoints for /predict and /plot

In [8]:
# 6. Create Flask API
print("\n6. Creating Flask API...")

app_content = '''"""
Flask API for AAPL Stock Price Prediction
"""
from flask import Flask, request, jsonify
import pickle
import numpy as np
import matplotlib.pyplot as plt
import io
import base64
import warnings
warnings.filterwarnings('ignore')

app = Flask(__name__)

# Load the trained model
try:
    with open('model/model.pkl', 'rb') as f:
        model = pickle.load(f)
    print("✓ Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

@app.route('/predict', methods=['POST'])
def predict():
    """POST endpoint for prediction with JSON features"""
    try:
        data = request.get_json()
        if not data:
            return jsonify({'error': 'No data provided'}), 400
        
        # Extract features
        features = data.get('features', [])
        if len(features) != 6:
            return jsonify({'error': 'Expected 6 features: [open, high, low, volume, close_ma_5, price_range]'}), 400
        
        # Validate feature types
        try:
            features = [float(f) for f in features]
        except ValueError:
            return jsonify({'error': 'All features must be numeric'}), 400
        
        # Make prediction
        if model is None:
            return jsonify({'error': 'Model not loaded'}), 500
        
        prediction = model.predict([features])[0]
        
        return jsonify({
            'prediction': float(prediction),
            'features': features,
            'model_info': {
                'coefficients': model.coef_.tolist(),
                'intercept': float(model.intercept_)
            }
        })
        
    except Exception as e:
        return jsonify({'error': f'Prediction failed: {str(e)}'}), 500

@app.route('/predict/<float:open_price>', methods=['GET'])
def predict_single(open_price):
    """GET endpoint for single feature prediction"""
    try:
        if open_price < 0 or open_price > 1:
            return jsonify({'error': 'Open price must be between 0 and 1'}), 400
        
        # Use default values for other features
        features = [open_price, open_price + 0.1, open_price - 0.1, 0.5, open_price, 0.2]
        
        if model is None:
            return jsonify({'error': 'Model not loaded'}), 500
        
        prediction = model.predict([features])[0]
        
        return jsonify({
            'prediction': float(prediction),
            'open_price': open_price,
            'default_features_used': features
        })
        
    except Exception as e:
        return jsonify({'error': f'Prediction failed: {str(e)}'}), 500

@app.route('/predict/<float:open_price>/<float:high_price>', methods=['GET'])
def predict_two(open_price, high_price):
    """GET endpoint for two feature prediction"""
    try:
        if open_price < 0 or open_price > 1 or high_price < 0 or high_price > 1:
            return jsonify({'error': 'Prices must be between 0 and 1'}), 400
        
        if high_price <= open_price:
            return jsonify({'error': 'High price must be greater than open price'}), 400
        
        # Use default values for other features
        low_price = open_price - 0.1
        features = [open_price, high_price, low_price, 0.5, open_price, high_price - low_price]
        
        if model is None:
            return jsonify({'error': 'Model not loaded'}), 500
        
        prediction = model.predict([features])[0]
        
        return jsonify({
            'prediction': float(prediction),
            'open_price': open_price,
            'high_price': high_price,
            'default_features_used': features
        })
        
    except Exception as e:
        return jsonify({'error': f'Prediction failed: {str(e)}'}), 500

@app.route('/plot')
def plot():
    """GET endpoint to return a simple chart"""
    try:
        # Create a sample plot
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Sample data
        dates = pd.date_range('2023-01-01', periods=50, freq='D')
        prices = np.linspace(0.5, 0.8, 50) + np.random.normal(0, 0.02, 50)
        
        ax.plot(dates, prices, linewidth=2, color='blue', alpha=0.7)
        ax.set_title('Sample AAPL Stock Price Trend', fontsize=14, fontweight='bold')
        ax.set_xlabel('Date')
        ax.set_ylabel('Normalized Price')
        ax.grid(True, alpha=0.3)
        
        # Save plot to bytes
        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=100, bbox_inches='tight')
        buf.seek(0)
        img_bytes = base64.b64encode(buf.read()).decode('utf-8')
        plt.close(fig)
        
        return f'<img src="data:image/png;base64,{img_bytes}" style="max-width:100%; height:auto;"/>'
        
    except Exception as e:
        return jsonify({'error': f'Plot generation failed: {str(e)}'}), 500

@app.route('/health')
def health():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'model_loaded': model is not None,
        'endpoints': [
            'POST /predict',
            'GET /predict/<open_price>',
            'GET /predict/<open_price>/<high_price>',
            'GET /plot',
            'GET /health'
        ]
    })

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
'''

with open('app.py', 'w') as f:
    f.write(app_content)

print("✓ Created app.py with Flask API")


6. Creating Flask API...
✓ Created app.py with Flask API


### Create requirements.txt

In [9]:
# 7. Create requirements.txt
print("\n7. Creating requirements.txt...")

requirements_content = '''flask==2.3.3
pandas==2.0.3
numpy==1.24.3
scikit-learn==1.3.0
matplotlib==3.7.2
seaborn==0.12.2
requests==2.31.0
gunicorn==21.2.0
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements_content)

print("✓ Created requirements.txt")


7. Creating requirements.txt...
✓ Created requirements.txt


### Create README.md

In [10]:
# 8. Create README.md
print("\n8. Creating README.md...")

readme_content = '''# AAPL Stock Price Prediction API

## Project Overview and Objectives

This project provides a machine learning API for predicting Apple Inc. (AAPL) stock prices based on historical market data. The model uses linear regression to predict normalized closing prices using engineered features including price ranges, moving averages, and return-based indicators.

### Key Features
- **Linear Regression Model**: Trained on 2023 AAPL stock data
- **Feature Engineering**: Price range, 5-day moving average, return-based features
- **RESTful API**: Multiple endpoints for different prediction scenarios
- **Error Handling**: Comprehensive input validation and error responses
- **Visualization**: Chart generation endpoint for data visualization

## How to Rerun Scripts/Notebooks

### Prerequisites
```bash
pip install -r requirements.txt
```

### Data Preparation
1. Ensure `project/data/processed/aapl_2023_cleaned.csv` exists
2. Run the notebook cells in order to:
   - Load and clean data
   - Engineer features
   - Train the model
   - Save model to `model/model.pkl`

### Model Training
```python
# The model is automatically trained when running the notebook
# Features used: ['Open', 'High', 'Low', 'Volume', 'close_ma_5_prev', 'price_range']
# Target: Normalized Close price
```

## Assumptions, Risks, and Lifecycle Mapping

### Assumptions
- **Data Quality**: Historical data is accurate and representative
- **Market Conditions**: 2023 market conditions are relevant for future predictions
- **Feature Relationships**: Linear relationships exist between features and target
- **Data Leakage**: No future information is used in feature engineering

### Risks
- **Market Volatility**: Stock markets are inherently unpredictable
- **Model Overfitting**: Linear model may not capture complex market dynamics
- **Data Drift**: Market conditions may change, reducing model accuracy
- **Feature Availability**: Some features may not be available in real-time

### Lifecycle Mapping
1. **Data Collection**: Daily stock data from reliable sources
2. **Feature Engineering**: Calculate technical indicators and price metrics
3. **Model Training**: Linear regression on historical data
4. **Model Deployment**: Flask API for real-time predictions
5. **Monitoring**: Track prediction accuracy and model performance
6. **Retraining**: Periodic model updates with new data

## Instructions for Using APIs or Dashboards

### Starting the API
```bash
python app.py
```
The API will be available at `http://localhost:5000`

### API Endpoints

#### 1. POST /predict
Predict stock price using JSON features.

**Request:**
```json
{
    "features": [0.5, 0.6, 0.4, 0.3, 0.55, 0.2]
}
```

**Response:**
```json
{
    "prediction": 0.523456,
    "features": [0.5, 0.6, 0.4, 0.3, 0.55, 0.2],
    "model_info": {
        "coefficients": [...],
        "intercept": 0.123
    }
}
```

#### 2. GET /predict/<open_price>
Predict using open price with default values for other features.

**Example:** `GET /predict/0.6`

#### 3. GET /predict/<open_price>/<high_price>
Predict using open and high prices with default values.

**Example:** `GET /predict/0.6/0.7`

#### 4. GET /plot
Generate and return a sample stock price chart.

#### 5. GET /health
Check API health and available endpoints.

### Error Handling
The API includes comprehensive error handling for:
- Missing or invalid input data
- Feature count mismatches
- Non-numeric features
- Model loading failures
- Invalid price ranges

### Testing the API
```python
import requests

# Test prediction endpoint
response = requests.post('http://localhost:5000/predict', 
                        json={'features': [0.5, 0.6, 0.4, 0.3, 0.55, 0.2]})
print(response.json())

# Test single feature endpoint
response = requests.get('http://localhost:5000/predict/0.6')
print(response.json())
```

## Project Structure
project/
├── app.py # Flask API application
├── model/
│ └── model.pkl # Trained model file
├── src/
│ └── utils.py # Utility functions
├── data/
│ └── processed/ # Processed data files
├── notebooks/ # Jupyter notebooks
├── requirements.txt # Python dependencies
└── README.md # This file


## Next Steps and Recommendations

### Short-term (1-3 months)
- Implement real-time data feeds
- Add more sophisticated models (Random Forest, Neural Networks)
- Create a web dashboard for interactive predictions
- Add model performance monitoring

### Medium-term (3-6 months)
- Implement ensemble methods for improved accuracy
- Add sentiment analysis from news and social media
- Create automated retraining pipelines
- Develop risk assessment metrics

### Long-term (6+ months)
- Expand to multiple stocks and markets
- Implement advanced time series models (LSTM, GRU)
- Add portfolio optimization features
- Create mobile applications

## Contact and Support
For questions or support, please refer to the project documentation or contact the development team.

---

**Disclaimer**: This model is for educational and research purposes only. Stock market predictions are inherently uncertain and should not be used as the sole basis for investment decisions. Always consult with financial professionals before making investment choices.
'''

with open('README.md', 'w') as f:
    f.write(readme_content)

print("✓ Created README.md")


8. Creating README.md...
✓ Created README.md


## 7. Testing the Flask API from Notebook

### TODO: Modify examples with your actual features

In [17]:
# 9. Test API from Notebook
print("\n9. Testing API from notebook...")

import requests
from IPython.display import display, HTML
import time

# Start Flask in background (simulated)
print("Starting Flask API...")
print("Note: In production, run 'python app.py' in a separate terminal")

# Test API endpoints (simulated)
print("\nTesting API endpoints:")

# Test POST /predict
print("1. Testing POST /predict...")
try:
    response = requests.post(
        'http://127.0.0.1:5000/predict',
        json={'features': [0.5, 0.6, 0.4, 0.3, 0.55, 0.2]},
        timeout=5
    )
    print(f"Response Status: {response.status_code}")
    print(f"Response Headers: {dict(response.headers)}")
    print(f"Response Text: {response.text}")
    
    # Try to parse JSON only if status is successful
    if response.status_code == 200:
        try:
            json_response = response.json()
            print(f"Response JSON: {json_response}")
        except requests.exceptions.JSONDecodeError:
            print("Response is not valid JSON")
    else:
        print(f"Request failed with status {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print("API not running. Start with: python app.py")
except Exception as e:
    print(f"Error: {e}")

# Test GET /predict/<input1>
print("\n2. Testing GET /predict/<input1>...")
try:
    response = requests.get('http://127.0.0.1:5000/predict/0.6', timeout=5)
    print(f"Response Status: {response.status_code}")
    print(f"Response Headers: {dict(response.headers)}")
    print(f"Response Text: {response.text}")
    
    if response.status_code == 200:
        try:
            json_response = response.json()
            print(f"Response JSON: {json_response}")
        except requests.exceptions.JSONDecodeError:
            print("Response is not valid JSON")
    else:
        print(f"Request failed with status {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print("API not running. Start with: python app.py")
except Exception as e:
    print(f"Error: {e}")

# Test GET /predict/<input1>/<input2>
print("\n3. Testing GET /predict/<input1>/<input2>...")
try:
    response = requests.get('http://127.0.0.1:5000/predict/0.6/0.7', timeout=5)
    print(f"Response Status: {response.status_code}")
    print(f"Response Headers: {dict(response.headers)}")
    print(f"Response Text: {response.text}")
    
    if response.status_code == 200:
        try:
            json_response = response.json()
            print(f"Response JSON: {json_response}")
        except requests.exceptions.JSONDecodeError:
            print("Response is not valid JSON")
    else:
        print(f"Request failed with status {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print("API not running. Start with: python app.py")
except Exception as e:
    print(f"Error: {e}")

# Test GET /plot
print("\n4. Testing GET /plot...")
try:
    response = requests.get('http://127.0.0.1:5000/plot', timeout=5)
    print(f"Response Status: {response.status_code}")
    print(f"Response Headers: {dict(response.headers)}")
    print(f"Response Content Type: {response.headers.get('content-type', 'unknown')}")
    print(f"Response Text Length: {len(response.text)} characters")
    print(f"Response Text Preview: {response.text[:200]}...")
    
    if response.status_code == 200:
        # Display the plot HTML if it's HTML content
        if 'text/html' in response.headers.get('content-type', ''):
            print("\nDisplaying plot HTML:")
            display(HTML(response.text))
        else:
            print(f"Full Response Text: {response.text}")
    else:
        print(f"Request failed with status {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print("API not running. Start with: python app.py")
except Exception as e:
    print(f"Error: {e}")

# Test GET /health
print("\n5. Testing GET /health...")
try:
    response = requests.get('http://127.0.0.1:5000/health', timeout=5)
    print(f"Response Status: {response.status_code}")
    print(f"Response Headers: {dict(response.headers)}")
    print(f"Response Text: {response.text}")
    
    if response.status_code == 200:
        try:
            json_response = response.json()
            print(f"Response JSON: {json_response}")
        except requests.exceptions.JSONDecodeError:
            print("Response is not valid JSON")
    else:
        print(f"Request failed with status {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print("API not running. Start with: python app.py")
except Exception as e:
    print(f"Error: {e}")

print("\nAPI testing complete!")
print("To run the API: python app.py")


9. Testing API from notebook...
Starting Flask API...
Note: In production, run 'python app.py' in a separate terminal

Testing API endpoints:
1. Testing POST /predict...
Response Status: 404
Response Headers: {'Server': 'Werkzeug/3.1.3 Python/3.13.2', 'Date': 'Wed, 27 Aug 2025 21:12:08 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '207', 'Connection': 'close'}
Response Text: <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

Request failed with status 404

2. Testing GET /predict/<input1>...
Response Status: 200
Response Headers: {'Server': 'Werkzeug/3.1.3 Python/3.13.2', 'Date': 'Wed, 27 Aug 2025 21:12:08 GMT', 'Content-Type': 'application/json', 'Content-Length': '120', 'Connection': 'close'}
Response Text: {
  "features_used": [
    0.6,
    0.7,
    0.5,
    0.5,
    0.6,
    0.2
  ],
  "input": 0.6,
  "p

### Create startup script

In [11]:
# 10. Create startup script
print("\n10. Creating startup script...")

startup_content = '''#!/bin/bash
# Startup script for AAPL Stock Prediction API

echo "Starting AAPL Stock Prediction API..."

# Check if Python is installed
if ! command -v python3 &> /dev/null; then
    echo "Error: Python 3 is not installed"
    exit 1
fi

# Check if requirements are installed
if [ ! -f "requirements.txt" ]; then
    echo "Error: requirements.txt not found"
    exit 1
fi

# Install requirements
echo "Installing requirements..."
pip install -r requirements.txt

# Check if model exists
if [ ! -f "model/model.pkl" ]; then
    echo "Error: model.pkl not found. Please run the notebook first."
    exit 1
fi

# Start the API
echo "Starting Flask API on http://localhost:5000"
echo "Press Ctrl+C to stop"
python app.py
'''

with open('start_api.sh', 'w') as f:
    f.write(startup_content)

# Make executable (Unix-like systems)
import os
os.chmod('start_api.sh', 0o755)

print("✓ Created start_api.sh startup script")


10. Creating startup script...
✓ Created start_api.sh startup script


In [13]:
# 11. Final Verification
print("\n11. Final verification...")

# Check all required files exist
required_files = [
    'app.py',
    'model/model.pkl', 
    'requirements.txt',
    'README.md',
    'src/utils.py',
    'start_api.sh'
]

print("Checking required files:")
for file_path in required_files:
    if Path(file_path).exists():
        print(f"✓ {file_path}")
    else:
        print(f"✗ {file_path} - MISSING")

# Verify model can be loaded
try:
    with open('model/model.pkl', 'rb') as f:
        test_model = pickle.load(f)
    print(f"✓ Model loaded successfully (coefficients: {len(test_model.coef_)})")
except Exception as e:
    print(f"✗ Model loading failed: {e}")

print("\n=== STAGE 13 COMPLETE ===")
print("Your AAPL Stock Prediction API is ready!")
print("\nTo start the API:")
print("1. python app.py")
print("2. Or: ./start_api.sh")
print("\nAPI will be available at: http://localhost:5000")
print("Test endpoints: /health, /predict, /plot")


11. Final verification...
Checking required files:
✓ app.py
✓ model/model.pkl
✓ requirements.txt
✓ README.md
✓ src/utils.py
✓ start_api.sh
✓ Model loaded successfully (coefficients: 6)

=== STAGE 13 COMPLETE ===
Your AAPL Stock Prediction API is ready!

To start the API:
1. python app.py
2. Or: ./start_api.sh

API will be available at: http://localhost:5000
Test endpoints: /health, /predict, /plot


## 9. Handoff Best Practices

- Ensure README.md is complete and clear
- Provide `requirements.txt` for reproducibility
- Ensure pickled model and scripts are in correct folders
- Verify another user can run the project end-to-end on a fresh environment